<a href="https://colab.research.google.com/github/rafiqhasan/AI_DL_ML_Repo/blob/master/All%20TFX/Vertex%20AI/Vertex_Pipe_%7C_Taxi_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kfp
!pip install google-cloud-pipeline-components

     |████████████████████████████████| 231 kB 16.1 MB/s 
     |████████████████████████████████| 127 kB 60.8 MB/s 
     |████████████████████████████████| 636 kB 68.3 MB/s 
     |████████████████████████████████| 105 kB 58.2 MB/s 
     |████████████████████████████████| 1.7 MB 50.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 5.2 MB/s 
     |████████████████████████████████| 75 kB 3.8 MB/s 
     |████████████████████████████████| 93 kB 1.3 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
  Created wheel for kfp: filename=kfp-1.7.0-py3-none-any.whl size=320984 sha256=07d3d5708467b5ab22d3e68e698728c28eb602105733ed928b31db27d833ec63
  Stored in directory: /root/.cache/pip/wheels/

     |████████████████████████████████| 81 kB 6.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 105 kB 52.1 MB/s 
     |████████████████████████████████| 104 kB 56.2 MB/s 
     |████████████████████████████████| 104 kB 57.3 MB/s 
     |████████████████████████████████| 103 kB 61.2 MB/s 
     |████████████████████████████████| 103 kB 48.9 MB/s 
     |████████████████████████████████| 103 kB 58.9 MB/s 
     |████████████████████████████████| 103 kB 60.8 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 97 kB 5.9 MB/s 
     |████████████████████████████████| 96 kB 5.2 MB/s 
     |████████████████████████████████| 96 kB 5.5 MB/s 
     |████████████████████████████████| 96 kB 5.6 MB/s 
     |████████████████████████████████| 92 kB 9.7 MB/s 
     |██████████████████████████

In [ ]:
import json
import logging
from google.colab import auth
from typing import NamedTuple

import kfp
# from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import dsl
from kfp.v2.dsl import (ClassificationMetrics, Input, Metrics, Model, Output,
                        component)
from kfp.v2.google.client import AIPlatformClient

auth.authenticate_user()

PROJECT_ID = 'us-gcp-ame-con-06f-npd-1'
DISPLAY_NAME = 'hasan-taxi-automl-pipeline'
REGION = 'us-central1'
PIPELINE_ROOT = 'gs://rafiqhasandttl_01/ml_pipelines/'
TRAIN_FILE = 'gs://rafiqhasandttl_01/taxi_dataset_withheader/train.csv'

In [ ]:
@kfp.dsl.pipeline(name="hasan-taxi-vertex-pipeline", pipeline_root=PIPELINE_ROOT)
def pipeline(
    display_name: str = DISPLAY_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    api_endpoint: str = "us-central1-aiplatform.googleapis.com"
):
    dataset_create_op = gcc_aip.TabularDatasetCreateOp( 
        project=project, display_name=display_name, gcs_source=TRAIN_FILE
    )

    #For non Auto-ML call
    # training_op = gcc_aip.CustomPythonPackageTrainingJobRunOp()

    #For AutoML
    training_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name=display_name,
        optimization_prediction_type="regression",
        optimization_objective="minimize-rmse", # binary classification        
        budget_milli_node_hours=750,
        dataset=dataset_create_op.outputs["dataset"],
        target_column="fare",
    )

    deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
        model=training_op.outputs["model"],
        project=project,
        machine_type="n1-standard-4",
    )

In [ ]:
!rm -f /content/hasan-taxi-vertex-pipeline.json

In [ ]:
from kfp.v2 import compiler  # noqa: F811

compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="/content/hasan-taxi-vertex-pipeline.json"
)

In [ ]:
from kfp.v2.google.client import AIPlatformClient  # noqa: F811

api_client = AIPlatformClient(project_id=PROJECT_ID, region=REGION)

/usr/local/lib/python3.7/dist-packages/kfp/v2/google/client/client.py:175: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


In [ ]:
response = api_client.create_run_from_job_spec(
    "/content/hasan-taxi-vertex-pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"project": PROJECT_ID, "display_name": DISPLAY_NAME},
)